## 상대 모멘텀
- 여러개의 종목을 이용하여 투자를 전략
    1. 월초부터 월말까지의 수정 종가를 이용하여 수익율 계산
    2. 월별 수익율이 높은 n개를 선택 
    3. 해당하는 종목들을 매수하여 해장하는 종목의 신호가 없어지면 매도하는 형식으로 수익율 생성

In [5]:
import pandas as pd 
import numpy as np 
import warnings
import os 
from glob import glob
from datetime import datetime

In [3]:
# 경고 메시지 출력 필터 
warnings.filterwarnings('ignore')

In [6]:
# 기준 년월 컬럼과 월별 수익율 컬럼을 생성하는 함수
# 매개변수 5개 
    # 데이터프레임 : 필수
    # ticker : 필수
    # 투자의 시작시간 : 2010-01-01 기본값
    # 투자의 종료시간 : 현재 시간 기본값
    # 기준이 되는 컬럼 : 'Adj Close' 기본값
def create_1m_rtn(
    _df, 
    _ticker, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _col = 'Adj Close'
):
    # 복사본 생성 
    df = _df.copy()
    # 'Date'가 컬럼에 존재하면 인덱스로 변환
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열 변환
    df.index = pd.to_datetime(df.index)
    # 투자의 시작시간과 종료시간으로 인덱스 필터링
    # 기준이되는 컬럼으로 컬럼 필터링 
    df = df.loc[_start : _end, [_col]]
    # 기준년월 컬럼을 생성해서 index에서 년도-월 추출하여 대입
    df['STD-YM'] = df.index.strftime('%Y-%m')
    # 월별 수익율 컬럼을 생성 
    df['1m_rtn'] = 0
    # ticker를 컬럼에 대입 
    df['CODE'] = _ticker
    # 기준년월의 유니크값을 생성
    ym_list = df['STD-YM'].unique()
    return df, ym_list



In [7]:
aapl = pd.read_csv("data/AAPL.csv")

In [8]:
aapl_df, ym_list = create_1m_rtn(aapl, 'AAPL')

In [ ]:
aapl_df

In [ ]:
ym_list

In [16]:
aapl_df.loc[ym_list[1], ].iloc[-1, 0] / \
    aapl_df.loc[ym_list[1], ].iloc[0, 0]

1.0507885741373026

In [ ]:
# 특정 위치의 파일의 목록을 로드 하는 기능 
os.listdir('./data')

In [24]:
files = glob("./data/*.csv")

- 반복문을 이용 
1. data 폴더에 있는 파일들을 로드
2. create_1m_rtn() 함수에 대입 ( 데이터프레임, 파일명 )
    - price_df, ym_list 변수에 대입
3. price_df를 새로운 데이터프레임(stock_df)에 단순 행 결합  
4. 기준년월의 유니크 값을 이용하여 반복문 생성
    - 기준년월의 데이터를 이용하여 인덱스 필터를 하여 월초, 월말의 데이터를 이용해서 월별 수익율 계산
    - price_df에 월별 수익율을 대입
    - price_df에 월말의 행을 새로운 데이터프레임(month_last_df) 단순 행결합

In [26]:
# 비어있는 데이터프레임 생성 
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

# files를 이용해서 반복문을 생성 
for file in files:
    # file : 특정 경로와 파일명
    # print(file)
    # 경로 파일명을 나눠준다. 
    folder, name = os.path.split(file)
    # 파일명에서 이름과 확장자로 나눠준다. 
    head, tail = os.path.splitext(name)
    # head는 create_1m_rtn에서 _ticker 매개변수에 넣어줄 인자값

    # 데이터프레임 로드 
    read_df = pd.read_csv(file)
    # 함수 호출 
    price_df, ym_list = create_1m_rtn(read_df, head)
    # price_df를 stock_df에 단순 행 결합
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 두번째 반복문 생성 
    # 월별 수익율을 계산하여 대입
    for ym in ym_list:
        # ym : 기준년월
        # 월초의 가격 (매수)
        buy = price_df.loc[ym, ].iloc[0, 0]
        # 월말의 가격 (매도)
        sell = price_df.loc[ym, ].iloc[-1, 0]
        # 수익율 생성
        rtn = sell / buy
        # 수익율 대입 
        price_df.loc[ym, '1m_rtn'] = rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합 
        last_df = price_df.loc[ym, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat(
            [month_last_df, last_df], axis=0)
            



In [28]:
month_last_df['CODE'].unique()

array(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY',
       'USM'], dtype=object)

In [29]:
# 백업 
month_rtn_df = month_last_df.copy()

In [31]:
# 인덱스를 리셋 
month_rtn_df.reset_index(inplace=True)

In [34]:
# 테이블을 재구조화 
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns = 'CODE', 
    values = '1m_rtn'
)

In [38]:
month_rtn_df.head(1)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.897435,0.936595,1.011692,0.85349,0.965027,NaN,0.910501,0.922809,0.947586,0.870922


In [43]:
# month_rtn_df의 데이터들을 랭크화(열의 값들은 이용)
month_rtn_df = month_rtn_df.rank(axis=1, 
    ascending=False, 
    pct=True)

In [45]:
# 상위의 40% 종목을 선택 
# where() 함수를 사용 
    # where (조건식, 거짓일때 대입된 데이터)
month_rtn_df = \
    month_rtn_df.where( month_rtn_df <= 0.4, 0 )

In [47]:
# 0이 아닌 데이터들은 1로 변환
month_rtn_df[month_rtn_df != 0] =1

In [50]:
month_rtn_df.head(1)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [49]:
# stock_df의 code의 unique()를 변수에 저장 
stock_codes = stock_df['CODE'].unique()

In [55]:
flag_col = month_rtn_df.loc['2010-01-29',] == 1
month_rtn_df.loc['2010-01-29' , flag_col].index

Index(['BND', 'GLD', 'SPY'], dtype='object', name='CODE')

In [56]:
# 해당 일자의 구매하려는 종목들을 딕셔너리 생성 
sig_dict = dict()

for idx in month_rtn_df.index:
    # idx : month_rtn_df의 인덱스들 (시계열)
    flag_col = month_rtn_df.loc[idx, ] == 1
    ticker_list = list(
        month_rtn_df.loc[idx, flag_col].index
    )
    # sig_dict에 추가
    sig_dict[idx] = ticker_list

In [ ]:
sig_dict

In [58]:
# stock_df를 재구조화 
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = 'Date', 
    columns = 'CODE', 
    values = stock_df.columns[0]
)

In [ ]:
stock_c_matrix.head()

In [60]:
# 거래내역 컬럼을 추가하는 함수 생성 
def create_trade_book(_df):
    # 복사본 생성 
    df = _df.copy()
    # 컬럼의 목록을 list로 생성 
    codes = list(df.columns)
    for code in codes:
        df[f"p_{code}"] = ""
        df[f"r_{code}"] = ""
    return df

In [61]:
book = create_trade_book(stock_c_matrix)

In [ ]:
book.head(2)

In [ ]:
# sig_dict을 이용해서 구매 대기 상태 입력 


In [64]:
for i in sig_dict:
    print(i)
    print(sig_dict[i])
    break

2010-01-29 00:00:00
['BND', 'GLD', 'SPY']


In [67]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND', 'GLD', 'SPY']), (Timestamp('2010-02-26 00:00:00'), ['AAPL', 'GDX', 'SPY']), (Timestamp('2010-03-31 00:00:00'), ['AAPL', 'AMZN', 'USM']), (Timestamp('2010-04-30 00:00:00'), ['AAPL', 'GDX', 'MSFT']), (Timestamp('2010-05-28 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-06-30 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-07-30 00:00:00'), ['MSFT', 'SPY', 'USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX', 'GLD', 'SLV']), (Timestamp('2010-09-30 00:00:00'), ['AAPL', 'AMZN', 'SLV']), (Timestamp('2010-10-29 00:00:00'), ['AMZN', 'MSFT', 'SLV']), (Timestamp('2010-11-30 00:00:00'), ['AMZN', 'GDX', 'GLD', 'SLV']), (Timestamp('2010-12-31 00:00:00'), ['GM', 'MSFT', 'SLV', 'USM']), (Timestamp('2011-01-31 00:00:00'), ['AAPL', 'BND', 'MSFT', 'SPY']), (Timestamp('2011-02-28 00:00:00'), ['AAPL', 'GDX', 'GLD', 'SLV']), (Timestamp('2011-03-31 00:00:00'), ['AMZN', 'SLV', 'SPY', 'USM']), (Timestamp('2011-04-29 00:00:00'), ['AMZN', 'GDX', 'G

In [69]:
for i, j in sig_dict.items():
    # i = sig_dict.items()[0]
    print(i)
    print(j)
    break

2010-01-29 00:00:00
['BND', 'GLD', 'SPY']
